# Meziroční analyza DPNK: 2017 → 2018

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import json

uaprev = json.load(open("UserAttendance-2017.json.anon"))
uacur = json.load(open("UserAttendance-2018.json.anon"))
prev = "2017"
cur = "2018"
print("Počet platicích učastníků ", prev,": ", len(uaprev))
print("Počet platicích učastníků ", cur,": ", len(uacur))
growth = len(uacur) - len(uaprev)
print("Meziroční růst: ", growth, "(", round(100*(len(uacur)/len(uaprev) - 1)), "%)")
print("Předpověd přištiho roku se stejným růstem:", round(len(uacur)*(len(uacur)/len(uaprev))))

Kolik lidí přihlašili se stejnou emailovou adresu?
--------------------------------------------------------

In [ ]:
def by_email(uas):
    d = {}
    for ua in uas:
        d[ua['userprofile__user__email']] = ua
    return d
by_emailprev = by_email(uaprev)
by_emailcur = by_email(uacur)
email_set_prev = set(by_emailprev.keys())
email_set_cur = set(by_emailcur.keys())
len(email_set_prev.intersection(email_set_cur))

Kolik lidí přihlašili se jinou (novou) emailovou adresu?
-----------------------------------------------------

In [ ]:
len(email_set_cur - email_set_prev)

Kolik lidí nejezdili pod stejnou emailovou adresu?
--------------------------------------------------

In [ ]:
len(email_set_prev - email_set_cur)

## Firmy

In [ ]:
def by_company(uas):
    d = {}
    for ua in uas:
        if not ua['team__subsidiary__company__name'] in d:
            d[ua['team__subsidiary__company__name']] = []
        d[ua['team__subsidiary__company__name']].append(ua)
    return d
by_firmprev = by_company(uaprev)
by_firmcur = by_company(uacur)
firm_set_prev = set(by_firmprev.keys())
firm_set_cur = set(by_firmcur.keys())
print("Počet firem ", prev, ": ", len(firm_set_prev))
print("Počet firem ", cur, ": ", len(firm_set_cur))
print("Průměrný počet učastniku na firmu ", prev, ": ", round(len(uaprev)/len(firm_set_prev), 2))
print("Průměrný počet učastniku na firmu ", cur, ": ", round(len(uacur)/len(firm_set_cur), 2))

### Největší firmy

In [ ]:
%matplotlib inline

In [ ]:
def plot_groups(groups, title):
    rider_counts = []
    for name, riders in groups.items():
        rider_counts.append((name, len(riders)))
    group_sizes = sorted(rider_counts, key=lambda t: t[1]) 
    import itertools
    group_names = []
    group_size = []
    for n, c in itertools.islice(reversed(group_sizes), 40):
        group_names.append(n)
        group_size.append(c)
    s = pd.Series(group_size, index=group_names)
    s.plot.bar(figsize=(10,10))
    plt.title(title);
plot_groups(by_firmcur, cur)

In [ ]:
plot_groups(by_firmprev, prev)

### Kolik firem s námi zůstali

In [ ]:
len(firm_set_prev.intersection(firm_set_cur))

### Kolik firem odešli?

In [ ]:
len(firm_set_prev - firm_set_cur)

### Kolik nových firem přibili?

In [ ]:
len(firm_set_cur - firm_set_prev)

### Největší firmy které odešli
Pozn: Je možné, že některé firmy se přejmenovali?

In [ ]:
lost_firms = {}
for firm in firm_set_prev - firm_set_cur:
    lost_firms[firm] = by_firmprev[firm]
plot_groups(lost_firms, "Odešli")

### Největší nové firmy
Pozn: Je možné, že některé firmy se přejmenovali?

In [ ]:
new_firms = {}
for firm in firm_set_cur - firm_set_prev:
    new_firms[firm] = by_firmcur[firm]
plot_groups(new_firms, "Nové velké firmy")

### Ztěch firem které snámi zůstali, jak vyvijeli počet učastniků, které sníma jezdili?

In [ ]:
difference_pairs = []
for firm in firm_set_prev.intersection(firm_set_cur):
    difference_pairs.append((firm,len(by_firmcur[firm]) - len(by_firmprev[firm])))
differences = []
firms = []
for f,d in sorted(difference_pairs, key=lambda a:a[1]):
    firms.append(f)
    differences.append(d)
print("Počet firem se stejným počtem účasniků:", len([x for x in differences if x == 0]))
print("Počet firem se větším počtem účasniků:", len([x for x in differences if x > 0]))
print("Počet firem se menším počtem účasniků:", len([x for x in differences if x < 0]))
s = pd.Series(itertools.islice(differences, 40), index=itertools.islice(firms,4O))
s.plot.bar(figsize=(10,25))
plt.title("Změny ve velikost firem (nejhorší)");

## Města

In [ ]:
def by_city(uas):
    d = {}
    for ua in uas:
        if not ua['team__subsidiary__city__name'] in d:
            d[ua['team__subsidiary__city__name']] = []
        d[ua['team__subsidiary__city__name']].append(ua)
    return d
by_cityprev = by_city(uaprev)
by_citycur = by_city(uacur)
city_set_prev = set(by_cityprev.keys())
city_set_cur = set(by_citycur.keys())
print("Počet měst", prev, ":", len(city_set_prev))
print("Počet měst", cur, ":", len(city_set_cur))
print("Průměrný počet učastniku na město", prev, ":", round(len(uaprev)/len(city_set_prev), 2))
print("Průměrný počet učastniku na město", cur, ":", round(len(uacur)/len(city_set_cur), 2))

### Největší města

In [ ]:
plot_groups(by_citycur, cur)

In [ ]:
plot_groups(by_cityprev, prev)

### Ztěch měst které měli učastnici oba roky, jak vyvijeli počet učastniků, které v ních jezdili?

In [ ]:
differences = []
for city in city_set_prev.intersection(city_set_cur):
    differences.append(len(by_citycur[city]) - len(by_cityprev[city]))
print("Počet měst se stejným počtem účasniků:", len([x for x in differences if x == 0]))
print("Počet měst se větším počtem účasniků:", len([x for x in differences if x > 0]))
print("Počet měst se menším počtem účasniků:", len([x for x in differences if x < 0]))

#### Které města měli menší počet učastníku?

In [ ]:
print("Ubyilo počet učastniků bez města")
for city in city_set_prev.intersection(city_set_cur):
    difference = len(by_citycur[city]) - len(by_cityprev[city])
    if difference < 0:
        print(city)
        print(" ", prev, ":", len(by_cityprev[city]))
        print(" ", cur, ":", len(by_citycur[city]))

### Růst města

In [ ]:
s = pd.Series(diferences, index=company_names)
s.plot.bar(figsize=(10,25))
plt.title("Růst");